In [6]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install -q sentencepiece


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
!nvidia-smi

Thu Jun 29 16:51:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10          On   | 00000000:06:00.0 Off |                    0 |
|  0%   47C    P0    60W / 150W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%env LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, LlamaTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, prepare_model_for_kbit_training
import re

model_name = "openlm-research/open_llama_7b"
tokenizer_name = "openlm-research/open_llama_7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map='auto',
    # quantization_config=bnb_config
)

# if re.match("[^/]+/llama", model_name):
print(f"Setting special tokens for LLaMA model {model_name}...")
model.config.pad_token_id = 0
model.config.bos_token_id = 1
model.config.eos_token_id = 2

print(f"Loaded model {model_name}")

tokenizer = LlamaTokenizer.from_pretrained("openlm-research/open_llama_7b")

# if re.match("[^/]+/llama", tokenizer_name):
print(
    f"Setting special tokens for LLaMA tokenizer {tokenizer_name}...")
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2

tokenizer.padding_side = "left"  # Allow batched inference

print(f"Loaded tokenizer {tokenizer_name}")

env: LD_LIBRARY_PATH=/usr/lib/x86_64-linux-gnu

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/lib/x86_64-linux-gnu/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting special tokens for LLaMA model openlm-research/open_llama_7b...
Loaded model openlm-research/open_llama_7b
Setting special tokens for LLaMA tokenizer openlm-research/open_llama_7b...
Loaded tokenizer openlm-research/open_llama_7b


In [3]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 8388608 || all params: 6746804224 || trainable%: 0.12433454005023165


In [6]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-6e72855d06356857/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-411cab03ae27b36c.arrow


In [7]:
print(f"Setting special tokens for LLaMA model {model_name}...")
model.config.pad_token_id = 0
model.config.bos_token_id = 1
model.config.eos_token_id = 2

print("Setting tokenizer tokens")
tokenizer.pad_token_id = 0
tokenizer.bos_token_id = 1
tokenizer.eos_token_id = 2

Setting special tokens for LLaMA model openlm-research/open_llama_7b...
Setting tokenizer tokens


In [8]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,2.100100
2,2.044600
3,1.589800
4,2.402700
5,2.195400
6,2.158900
7,2.397000
8,2.349000
9,2.570700
10,1.908000


TrainOutput(global_step=200, training_loss=1.4184346938133239, metrics={'train_runtime': 938.2024, 'train_samples_per_second': 3.411, 'train_steps_per_second': 0.213, 'total_flos': 1.1841155089661952e+16, 'train_loss': 1.4184346938133239, 'epoch': 1.28})

In [10]:
print(dir(trainer))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_sm_patterns_to_gitignore', '_gather_and_numpify', '_get_collator_with_removed_columns', '_get_eval_sampler', '_get_learning_rate', '_get_output_dir', '_get_train_sampler', '_globalstep_last_logged', '_hp_search_setup', '_inner_training_loop', '_issue_warnings_after_load', '_load_best_model', '_load_from_checkpoint', '_load_optimizer_and_scheduler', '_load_rng_state', '_loggers_initialized', '_maybe_log_save_evaluate', '_memory_tracker', '_move_model_to_device', '_nested_gather', '_pad_across_processes', '_prepare_input', '_prepare_inputs', '_push_from_checkpoint', '_remove_unused_columns', '_report_to_hp_search', '_rotate_checkpoints', '_sav

In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
trainer.save_model("quotes_test")

In [18]:
model.push_to_hub("Nevin1901/quotes_test")

CommitInfo(commit_url='https://huggingface.co/Nevin1901/quotes_test/commit/7da80e20c1f61b5ca8d20db81651fc5c4886e49c', commit_message='Upload model', commit_description='', oid='7da80e20c1f61b5ca8d20db81651fc5c4886e49c', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
# Inference
%pip install protobuf==3.20.*

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "./quotes_test/"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /usr/bin/python3: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 00
CUDA SETUP: Loading binary /hom

/home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/run/lambda-jupyter-lab.pid')}
  warn(msg)
/home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//ipykernel.pylab.backend_inline')}
  warn(msg)
/home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)
The model weights are not tied. Please use the `tie_weights` method b

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# Inference
batch = tokenizer("I am so clever", return_tensors='pt').to("cuda")

with torch.cuda.amp.autocast():
  output = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output[0], skip_special_tokens=True))

/home/ubuntu/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/lib/python3/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")




 I am so clever. I am so smart. I am so very, very clever.
I am so clever. I am so smart. I am so very, very clever.
I am so clever. I am so smart. I am so very, very
